# Ad Spend Optimization: Data Cleaning & SQL Analysis
**Author:** Mariana Saca

**Goal:** Transform raw social media campaign data into a clean dataset for ROI prediction and use SQL to uncover high-level strategic insights.

### Phase 1: Data Cleaning
The raw data contains mixed types (strings with currency symbols) and merged columns (Target Audience) that need to be parsed before analysis.

In [1]:
import pandas as pd
import sqlite3
import numpy as np

# Load the raw dataset
df = pd.read_csv('../data/Social_Media_Advertising.csv')

# Initial check
print(f"Initial Shape: {df.shape}")
df.head(3)

Initial Shape: (300000, 16)


,Campaign_ID,Target_Audience,Campaign_Goal,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,Company
0,529013,Men 35-44,Product Launch,15 Days,Instagram,0.15,$500.00,5.79,Las Vegas,Spanish,500,3000,7,Health,2022-02-25,Aura Align
1,275352,Women 45-60,Market Expansion,15 Days,Facebook,0.01,$500.00,7.21,Los Angeles,French,500,3000,5,Home,2022-05-12,Hearth Harmony
2,692322,Men 45-60,Product Launch,15 Days,Instagram,0.08,$500.00,0.43,Austin,Spanish,500,3000,9,Technology,2022-06-19,Cyber Circuit


### 1. Cleaning "Dirty" Columns
The `Acquisition_Cost` and `Duration` columns are currently objects (strings) because they contain symbols ('$') and text ('Days'). We need to convert these to numeric types for calculation.

In [4]:
# 1. Clean Acquisition_Cost: Remove '$' and convert to float
# errors='coerce' turns unparseable data into NaN (safe handling)
df['Acquisition_Cost'] = df['Acquisition_Cost'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
df['Acquisition_Cost'] = pd.to_numeric(df['Acquisition_Cost'], errors='coerce')

# 2. Clean Duration: Remove ' Days' and convert to integer
df['Duration'] = df['Duration'].astype(str).str.replace(' Days', '', regex=False)
df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')

# Check the new data types
print("New Data Types:")
print(df[['Acquisition_Cost', 'Duration']].dtypes)

New Data Types:
Acquisition_Cost    float64
Duration              int64
dtype: object


### 2. Feature Engineering (The "Strategy" Layer)
The `Target_Audience` column combines two critical variables: **Gender** and **Age Group** (e.g., "Men 35-44"). To optimize targeting strategy, we must split these into distinct features.

In [5]:
# Split 'Target_Audience' into two new columns
# We use str.split() with expand=True to create a new dataframe of columns
df[['Target_Gender', 'Target_Age_Group']] = df['Target_Audience'].str.split(' ', n=1, expand=True)

# Validate the split
print(df[['Target_Audience', 'Target_Gender', 'Target_Age_Group']].head(3))

# Drop the original redundant column to keep things clean
df.drop(columns=['Target_Audience'], inplace=True)

  Target_Audience Target_Gender Target_Age_Group
0       Men 35-44           Men            35-44
1     Women 45-60         Women            45-60
2       Men 45-60           Men            45-60


### 3. SQL Analysis Layer
Instead of only using Pandas, we will inject the data into a local SQLite database. This allows us to run complex SQL queries (Window Functions, CTEs) to answer business questions directly.

In [6]:
# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Write the cleaned dataframe to the SQL database
df.to_sql('campaigns', conn, index=False, if_exists='replace')

print("Database initialized. Table 'campaigns' created.")

Database initialized. Table 'campaigns' created.


#### **Business Question 1: Efficiency by Channel**
Which channel delivers the best ROI stability? We will calculate the average ROI and the Standard Deviation (Risk) for each channel.

In [7]:
query_efficiency = """
SELECT 
    Channel_Used,
    COUNT(Campaign_ID) as Total_Campaigns,
    ROUND(AVG(ROI), 2) as Avg_ROI,
    ROUND(AVG(Conversion_Rate), 3) as Avg_Conversion
FROM campaigns
GROUP BY Channel_Used
ORDER BY Avg_ROI DESC;
"""

# Run query and display as dataframe
efficiency_df = pd.read_sql(query_efficiency, conn)
display(efficiency_df)

,Channel_Used,Total_Campaigns,Avg_ROI,Avg_Conversion
0,Instagram,75101,4.01,0.08
1,Twitter,74653,4.00,0.08
2,Facebook,75164,3.99,0.08
3,Pinterest,75082,0.72,0.08


#### **Business Question 2: Advanced Ranking (Window Functions)**
**Strategy:** Identify the top performing Campaign Goal for *each* Location based on Engagement Score.
**Technique:** Use `RANK()` partition by Location to find regional preferences.

In [9]:
query_ranking = """
WITH Regional_Stats AS (
    -- Step 1: Calculate Average Engagement per City & Goal
    SELECT 
        Location,
        Campaign_Goal,
        AVG(Engagement_Score) as Avg_Engagement
    FROM campaigns
    GROUP BY Location, Campaign_Goal
),
Ranked_Stats AS (
    -- Step 2: Assign the Rank (We cannot filter here yet!)
    SELECT 
        Location,
        Campaign_Goal,
        Avg_Engagement,
        RANK() OVER (PARTITION BY Location ORDER BY Avg_Engagement DESC) as Rank_Val
    FROM Regional_Stats
)
-- Step 3: Now we can filter because Rank_Val was calculated in the previous step
SELECT 
    Location, 
    Campaign_Goal, 
    ROUND(Avg_Engagement, 2) as Avg_Engagement
FROM Ranked_Stats
WHERE Rank_Val = 1;
"""

ranking_df = pd.read_sql(query_ranking, conn)
display(ranking_df)

,Location,Campaign_Goal,Avg_Engagement
0,Austin,Product Launch,4.39
1,Las Vegas,Market Expansion,4.40
2,Los Angeles,Market Expansion,4.42
3,Miami,Increase Sales,4.41
4,New York,Brand Awareness,4.40


In [10]:
# 4. Save Processed Data
import os

# Create the folder if it doesn't exist
os.makedirs('../data/processed', exist_ok=True)

# Save the clean dataframe to a new CSV
df.to_csv('../data/processed/cleaned_ads_data.csv', index=False)

print("✅ Success! File saved to ../data/processed/cleaned_ads_data.csv")

✅ Success! File saved to ../data/processed/cleaned_ads_data.csv
